# OpenSearch 에 document 를 vector 로 변환해서 저장합니다.

document 는 chunk 단위로 나뉜 다음에 vector 로 변환되어 저장합니다.

## 0. 필요한 변수 불러오기

In [ ]:
# OpenSearch와 통신하기 위해서 앞서 저장했던 변수를 불러옵니다.
%store -r opensearch_user_id opensearch_user_password opensearch_domain_name opensearch_domain_endpoint

print(f"OpenSearch User Id: {opensearch_user_id}")
print(f"OpenSearch User Password: {opensearch_user_password}")
print(f"OpenSearch Domain Name: {opensearch_domain_name}")
print(f"OpenSearch Domain Endpoint: {opensearch_domain_endpoint}")
print()
    
    
# SageMaker Endpoint 와 통신하기 위해서 저장했던 변수를 불러옵니다.
%store -r endpoint_name
print(f"SageMaker EndpointName: {endpoint_name}")
print()

In [ ]:
# 저장이 되어 있지 않다면 직접 관련 정보를 입력합니다.

# opensearch_user_id = 'raguser'
# opensearch_user_password = 'Passw0rd1!'

# opensearch_domain_name = 'rag-hol-6efd6d14'
# opensearch_domain_endpoint = 'https://search-rag-hol-6efd6d14-5wbkv7qjrlci47h5ka63cw5fxy.us-west-2.es.amazonaws.com'

# endpoint_name='llama-3-2-1b-instruct-2024-11-18-14-18-13-972'

## 1. OpenSearch 호출 테스트 - 문서 저장

### 1-1. 필요한 함수 정의

- read_pdf(): PDF 파일 읽는 함수
- split_into_sentences(): 문서를 문장단위로 분리하는 함수
- split_into_chunk_docs(): 문서를 문장단위로 분리하고 다시 chunk 로 나누는 함수

In [ ]:
import re
import os
import PyPDF2
from datetime import datetime
from langchain_core.documents import Document


# PDF 읽는 함수 정의
def read_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    
    file_name = os.path.basename(file_path)
    type_name = file_name.split('_')[0]

    return text, file_name, type_name


# 문서를 문장으로 분리하는 함수 정의
def split_into_sentences(text):
    return re.split(r'(?<=[.!?])\s+', text)


# 문서를 chunk 단위로 나누는 함수 정의
def split_into_chunk_docs(text, file_name, type_name, chunk_size=1000):
    docs = []
    sentences = split_into_sentences(text)
    current_chunk = ""
    
    def make_document(chunk):
        return Document(
                page_content=chunk.strip().replace('\n', ' '),
                metadata={
                    "source": file_name,
                    "type": type_name,
                    "timestamp": datetime.now()
                }
            )
    
    for sentence in sentences:
        current_chunk += sentence + " "
        if len(current_chunk) >= chunk_size:
            docs.append(make_document(current_chunk))
            current_chunk = ""
    
    docs.append(make_document(current_chunk))
    
    return docs

### 1-2. PDF 문서를 OpenSearch에 넣기 위해서 Chunk 단위로 분리

In [ ]:
file_path = "file/Amazon-com-Inc-2023-Shareholder-Letter.pdf"
text, file_name, type_name = read_pdf(file_path)
documents = split_into_chunk_docs(text, file_name, type_name, chunk_size=800)

In [ ]:
print(f"Documents Size: {len(documents)}")
print("Documents Content:")
print(documents[0])
print()

### 1-3. OpenSearch 에 저장

In [ ]:
import boto3
from langchain_aws import BedrockEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch

# Embedding Model 생성
bedrock_runtime = boto3.client('bedrock-runtime')
embedding = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v2:0", # Titan Text Embeddings V2
)

# Index Name 정의
index_name = "rag-index-name"

# OpenSearch 인증 정보
http_auth = (opensearch_user_id, opensearch_user_password)

# OpenSearchVectorSearch 생성
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=embedding,
    http_auth=http_auth, # http_auth
)

In [ ]:
vector_db.add_documents(documents)

## 4. OpenSearch 호출 테스트 - 문서 검색

In [ ]:
query = "아마존의 비전은 무엇인가요"
k = 3
results=vector_db.similarity_search_with_score(
    query,
    k=k
)
page_contents = [res[0].page_content for res in results]
context = page_contents[0]
print(context)